# RG374 FACS analysis IFNAR fl LysMcre CpG

In [54]:
options(warn=-1)

In [55]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
                         
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel)

        )
    )
)

In [56]:
random_seed <- 42
set.seed(random_seed)

In [57]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [58]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter 

In [59]:
color$genotype <- c("D0 +/+"="#66c2a5", "D0 cre/+"="#00634A", "D3 +/+"="#cd34b5", "D3 cre/+"="#FFAC1E", "D6 +/+"="#cd34b5", "D6 cre/+"="#FFAC1E")

# Functions 

In [60]:
scale_label <- function(x, y) {
    
    max_y <- max(x[[y]])
    float_y <- attr(regexpr("(?<=\\.)0+", max_y, perl = TRUE), "match.length")
    
    if(max_y>=1) {
    
        x <- x
        ylab="10^6~x~cells/spleen"
        
        return(list(mat=x, ylab=ylab))

    } else if (float_y==-1) {
        
        x[[y]] <- x[[y]]*10
        ylab <- paste0("10^5~x~cells/spleen")

        return(list(mat=x, ylab=ylab)) 
    
    } else {

        exp_scale <- 1+float_y

        x[[y]] <- x[[y]]*10^exp_scale
        ylab <- paste0("10^", 6-exp_scale, "~x~cells/spleen")

        return(list(mat=x, ylab=ylab))

    }
        
}

In [61]:
bar_plot <- function(mat, x, y, xlab="", ylab="", ggtitle="", scale_y=FALSE, parse_y=FALSE, expand_y=NULL, position_bar="stack", position_jitter="jitter", position="default") {
    
    if(scale_y) {
        
        res <- scale_label(mat, deparse(substitute(y)))
        mat <- res[[1]]
        ylab <- res[[2]]

    }

    if(position=="dodge") {

        position_bar <- position_dodge(width=0.8)
        position_jitter <- position_dodge(width=0.8)
    }
    
    p <- ggplot(mat, aes(x={{x}}, y={{y}}, color=genotype, fill=genotype)) + 
    
        geom_bar(stat="summary", width=0.8, color="black", size=0.1, fun=mean, position=position_bar) + 
        geom_jitter(shape=21, stroke=0.1, size=1.0, show.legend=FALSE, color="black", position=position_jitter) + 
        xlab(xlab) + ylab(ylab) + ggtitle(ggtitle) + 
        scale_color_manual(values=color$genotype) + 
        scale_fill_manual(values=color$genotype) + 
        theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
        guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))
    
    if(parse_y) {
        
        p <- p + ylab(parse(text=ylab))
        
    }


    if(!is.null(expand_y)) {

         p <- p + ylim(0, (max(ggplot_build(p)$data[[1]]$y) + expand_y))
    }
    
    return(p)
    
}

# Import data 

In [62]:
sheet_names <- openxlsx::getSheetNames("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx")

In [63]:
data <- lapply(sheet_names, function(x) openxlsx::read.xlsx("data/RG374_ifnar_fl_lysmcre_cpg/facs/spleen//RG374_facs_spleen_data.xlsx", sheet=x))
names(data) <- sheet_names

## Spleen weight 

In [64]:
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -viable_cells) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [65]:
bar_plot_1 <- bar_plot(mat, genotype, spleen_weight_mg, ylab="Spleen weight [mg]", ggtitle="Spleen Weight") + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [66]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_spleen_weight.pdf", width=1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

## Viable cells 

In [42]:
mat <- data[["spleen_overview"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample, -spleen_weight_mg) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(viable_cells=viable_cells/10^4)

In [43]:
bar_plot_1 <- bar_plot(mat, genotype, viable_cells, ylab="10^4~viable~cells/spleen", parse_y=TRUE, ggtitle="Spleen viable cells") + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [44]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_viable_cells.pdf", width=1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

## CD44 SSC gating 

In [45]:
mat <- data[["cd44_ssc_gating"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("ratio")) 

In [46]:
bar_plot_1 <- bar_plot(mat, cell_type, ratio, ylab="10^6~viable~cells/spleen", parse_y=TRUE, position="dodge", ggtitle="Spleen CD44 SSC") + facet_grid(~time_point) + theme(legend.position="none") + theme_global_set(4)
bar_plot_1 <- egg::set_panel_size(bar_plot_1, width=unit(1.8, "cm"), height=unit(2.0, "cm"))

In [47]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_ssc_gating.pdf", width=3*1.5, height=1.9)

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

# Relative cell numbers 

In [48]:
mat <- data[["relative"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("ratio"))  

In [49]:
bar_plot_1 <- lapply(split(mat, f=mat$cell_type), function(x) bar_plot(x, genotype, ratio, ylab="Ratio [%]", ggtitle=x$cell_type[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [50]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_relative.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2

## Absolut numbers 

In [51]:
mat <- data[["absolut"]] %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6"))) %>%
    dplyr::mutate(genotype=paste(time_point, genotype)) %>% 
    dplyr::mutate(genotype=factor(genotype, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cell_type", value.name=c("ratio"))  

In [52]:
bar_plot_1 <- lapply(split(mat, f=mat$cell_type), function(x) bar_plot(x, genotype, ratio, ylab="Absolut", ggtitle=x$cell_type[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [53]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_1/bp_facs_absolut.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

pdf 
  2